# SageMaker - Distributed Data Parallelization with Tensorflow - Using Notebook

Used in situation where you have to train a model in a really big data set. So we distribute the data among multiple GPUs. 

**Setup**

In [ ]:
!pip install sagemaker --upgrade

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

**Training script**

In [ ]:
!pygmentize train_tensorflow_smdataparallel_mnist.py

**SageMaker Estimator**

 - **`Instance types`** - smdistributed.dataparallel
 
 - **`Instance count`** - To get the best performance and the most out of `smdistributed.dataparallel`, you should use at least 2 instances, but you can also use 1 for testing this example.
 
 - **`Distribution strategy`** - Note that to use DDP mode, you update the the distribution strategy, and set it to use smdistributed dataparallel.

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    base_job_name="tensorflow2-smdataparallel-mnist",
    #source_dir="code",
    entry_point="train_tensorflow_smdataparallel_mnist.py",
    role=role,
    py_version="py37",
    framework_version="2.4.1",
    
    # For training with multinode distributed training, set this count. Example: 1
    instance_count=2,
    
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.m4.xlarge",
    sagemaker_session=sagemaker_session,
    
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
)

In [ ]:
estimator.fit()

Now that you have a trained model, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests. The following cell will store the model_data variable to be used with the inference notebook.

In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

In [ ]:
print(predictor.endpoint_name)

In [ ]:
import tensorflow as tf
import numpy as np

(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data(path="/tmp/data")

In [ ]:
for i in range(10):
    data = mnist_images[i].reshape(1, 28, 28, 1)

    predict_response = predictor.predict(data)

    print("========================================")
    label = mnist_labels[i]

    predict_label = np.argmax(predict_response["predictions"])

    print("label is {}".format(label))
    print("prediction is {}".format(predict_label))

**Cleanup**

In [ ]:
predictor.delete_endpoint()